In [9]:
import cv2
import os
import numpy as np

Image Extraction and Processing


In [10]:
img = cv2.imread("./images/img_3.jpg")
img = cv2.resize(img, (450, 450))
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.GaussianBlur(img_gray, (5, 5), 1)
img_threshold = cv2.adaptiveThreshold(img_blur, 255, 1, 1, 11, 2)
output_path = os.path.join("images", "processed_image.jpg")
cv2.imwrite(output_path, img_threshold)

True

Contour Mapping


In [11]:
imgcontour = img.copy()
imgBigContour = img.copy()
contours, hierarchy = cv2.findContours(
    img_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(imgcontour, contours, -1, (0, 255, 0), 3)
cv2.imshow("Sudoku", imgcontour)
cv2.waitKey(0)
cv2.destroyAllWindows()

BiggestContour


In [12]:
def biggestContour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest, max_area

In [13]:
def reorder(points):
    points = points.reshape((4, 2))
    points_new = np.zeros((4, 1, 2), np.int32)
    add = points.sum(1)
    points_new[0] = points[np.argmin(add)]
    points_new[3] = points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    points_new[1] = points[np.argmin(diff)]
    points_new[2] = points[np.argmax(diff)]
    return points_new

In [14]:
biggest, maxArea = biggestContour(contours)
if biggest.size != 0:
    biggest = reorder(biggest)
    cv2.drawContours(imgBigContour, biggest, -1, (0, 255, 0), 20)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0, 0], [450, 0], [0, 450], [450, 450]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (450, 450))
    imgDetectedDigits = imgWarpColored.copy()
    imgWarpGray = cv2.cvtColor(imgWarpColored, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Sudoku", imgWarpColored)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [15]:
def splitBoxes(img):
    rows = np.vsplit(img, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

In [16]:
boxes = splitBoxes(imgWarpGray)
cv2.imshow("Sudoku", boxes[4])
cv2.waitKey(0)
cv2.destroyAllWindows()